In [1]:
from ultralytics import YOLO

In [6]:
model = YOLO("runs/detect/train/weights/best.pt")
results = model.predict("../source/test_video/OTV5.mp4", save=True)


output_dir = model.predictor.save_dir
print(f"Results saved to: {output_dir}")



WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/1247) d:\Fauzan\KULIAH\Semester 7\Computer Vision\repository-lab\repository_lab_cv\proyek_kelompok\script\..\source\test_video\OTV5.mp4: 640x640 1 pin magnet, 1 mario, 8.0ms
video 1/1 (frame 2/1247) d:\Fauzan\KULIAH\Semester 7\Computer Vision\repository-lab\repository_lab_cv\proyek_kelompok\script\..\source\test_video\OTV5.mp4: 640x640 1 pin magnet, 1 mario, 11.0ms
video 1/1 (frame 3/1247) d:\Fauzan\KULIAH\Semester 7\Computer Vision\re

In [2]:
from ultralytics import YOLO
import cv2

# Load the fine-tuned YOLO model
model = YOLO("runs/detect/train/weights/best.pt")  # Path ke model hasil fine-tune

# Video path or camera feed
video_path = "../source/test_video/OTV1.mp4"
output_path = "runs/detect/predict/OTV1_detected.mp4"

# Open the video
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, int(cap.get(cv2.CAP_PROP_FPS)),
                      (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

# Define class-specific colors
highlight_class_id = 1  # Class ID to highlight
highlight_color = (0, 255, 0)  # Green for highlighted class
default_color = (0, 0, 255)    # Red for others

# Progress bar configuration
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
processed_frames = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Inference on the frame (suppress verbose output)
    results = model.predict(frame, conf=0.25, verbose=False)  # Disable verbose output

    for box in results[0].boxes:  # Loop through bounding boxes
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())  # Bounding box coordinates
        conf = box.conf[0].item()  # Confidence score
        cls_id = int(box.cls[0].item())  # Class ID
        label = model.names[cls_id]  # Class name

        # Determine the color based on the class ID
        color = highlight_color if cls_id == highlight_class_id else default_color

        # Draw the bounding box and label
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Write the frame to the output video
    out.write(frame)

    # Update progress bar
    processed_frames += 1
    print(f"\rProcessing frame {processed_frames}/{total_frames}...", end="")

cap.release()
out.release()
cv2.destroyAllWindows()
print("\nProcessing complete.")


Processing frame 1008/1008...
Processing complete.
